Imports

In [ ]:
import json
import numpy as np
from gensim import corpora, models, similarities
from nltk.tokenize import word_tokenize
import nltk
import os
import shutilc
import logging

Carregar variáveis globais

In [ ]:
%store -r id_consulta

Configuração do Logger

In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

Carregando .jsons

In [ ]:
with open('map_documentos.json', 'r', encoding='utf-8') as f:
    map_documentos = json.load(f)
with open('map_consultas.json', 'r', encoding='utf-8') as f:
    map_consultas = json.load(f)

Download de dependências do NLTK


In [ ]:
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')

Indexar o map_documentos com Vetorial (TF-IDF)

In [ ]:
# Crie uma lista com os textos dos documentos. A ordem será preservada.
lista_de_documentos = list(map_documentos.values())
# Crie o corpus tokenizado a partir dessa lista
corpus_tokenizado = [doc.lower().split(" ") for doc in lista_de_documentos]
# Guardar os IDs dos documentos na mesma ordem para mapeamento posterior
ids_documentos = list(map_documentos.keys())

Construção do modelo Vetorial

In [ ]:
logging.info("Indexando o corpus com o modelo Vetorial (TF-IDF)...")
dictionary = corpora.Dictionary(corpus_tokenizado)
bow_corpus = [dictionary.doc2bow(text) for text in corpus_tokenizado]
tfidf_model = models.TfidfModel(bow_corpus)
tfidf_corpus = tfidf_model[bow_corpus]

# Usando 'similarities.Similarity' para eficiência de memória
temp_folder = './temp_vetorial_index'
if os.path.exists(temp_folder):
    shutil.rmtree(temp_folder)
os.makedirs(temp_folder)
output_prefix = os.path.join(temp_folder, 'shard')
index = similarities.Similarity(output_prefix, tfidf_corpus, num_features=len(dictionary))


Função que retorna Top N documentos de uma consulta Vetorial

In [ ]:
def obter_top_n_ids_vetorial(consulta_tokenizada, n):
    # 1. Converte a consulta para o espaço vetorial TF-IDF
    query_bow = dictionary.doc2bow(consulta_tokenizada)
    query_tfidf = tfidf_model[query_bow]

    # 2. Calcula os scores (similaridades) para cada documento no corpus
    doc_scores = index[query_tfidf]

    # 3. Pega os índices dos scores em ordem decrescente de pontuação
    top_n_indices = np.argsort(doc_scores)[::-1]

    # 4. Retorna apenas os 'n' primeiros IDs da lista ordenada
    # Pega o ID original da lista 'ids_documentos' e converte para inteiro
    return [int(ids_documentos[idx]) for idx in top_n_indices[:n]]

Executar consulta e gerar os resultados top 50

In [ ]:
# Escolha uma consulta para testar
consulta_texto = map_consultas[id_consulta]
consulta_tokenizada = consulta_texto.lower().split(" ")
# Chame a função, que retornará os IDs dos documentos mais relevantes.
resultados_vetorial = obter_top_n_ids_vetorial(consulta_tokenizada, n=50)

Salvando resultados_vetorial como .json

In [ ]:
nome_arquivo_saida = 'resultados_tfidf.json'
logging.info(f"Salvando resultados em '{nome_arquivo_saida}'...")
try:
    with open(nome_arquivo_saida, 'w', encoding='utf-8') as out_file_opened:
        json.dump(
            resultados_vetorial,    # Dados a serem salvos
            out_file_opened,        # Objeto do arquivo
            indent=2,               # Indentação para melhorar legibilidade
            ensure_ascii=False      # Permite caracteres não-ASCII
        )
    logging.info("Arquivo de resultados salvo com sucesso!")
except Exception as e:
    logging.error(f"Erro ao salvar o arquivo JSON: {e}")
finally:
    # Limpa a pasta temporária do índice
    if os.path.exists(temp_folder):
        shutil.rmtree(temp_folder)